In [1]:
import pandas as pd 
import numpy as np
import spacy

In [2]:
df=pd.read_csv("C:/Users/ankom/Downloads/twitter_training.csv/twitter_training.csv")

In [3]:
df.head()

,id,country,label,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       74682 non-null  int64 
 1   country  74682 non-null  object
 2   label    74682 non-null  object
 3   text     73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [5]:
df.isna().sum()

id           0
country      0
label        0
text       686
dtype: int64

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.isna().sum()

id           0
country      0
label        0
text       326
dtype: int64

In [8]:
df.shape

(71981, 4)

In [9]:
df.dropna(inplace=True)

In [10]:
df.shape

(71655, 4)

In [11]:
df.isna().sum()

id         0
country    0
label      0
text       0
dtype: int64

In [15]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 919.0 kB/s eta 0:00:14
      --------------------------------------- 0.2/12.8 MB 1.7 MB/s eta 0:00:08
     - -------------------------------------- 0.6/12.8 MB 3.5 MB/s eta 0:00:04
     ---- ----------------------------------- 1.3/12.8 MB 5.8 MB/s eta 0:00:02
     ----- ---------------------------------- 1.8/12.8 MB 6.7 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.8 MB 7.4 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/12.8 MB 8.0 MB/s eta 0:00:02
     ---------- ----------------------------- 3.5/12.8 MB 8.6 MB/s eta 0:00:02
     ------------ --------------------------- 3.9/12.8 MB 8.9 MB/s eta 0:00:02
     -------------- ------------------------- 4.7/12.8 MB 9.1 MB/s eta 0:00:01
     ---------------- ----------------------- 5.3/12.8 MB 9.7 MB/

In [16]:
nlp = spacy.load("en_core_web_sm")

In [17]:
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

In [19]:
df['Preprocessed Text'] = df['text'].apply(preprocess) 

In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
le_model = LabelEncoder()
df['label'] = le_model.fit_transform(df['label'])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df['Preprocessed Text'], df['label'],test_size=0.2, random_state=42, stratify=df['label'])

In [24]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (57324,)
Shape of X_test:  (14331,)


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)
y_pred = model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.63


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [30]:
rf = RandomForestClassifier()
rf.fit(X_train_vectorized, y_train)

RandomForestClassifier()

In [32]:
y_pred_rf = rf.predict(X_test_vectorized)

In [33]:
print("Accuracy:", accuracy_score(y_test, y_pred_rf))

Accuracy: 0.8844463052124765
